In [ ]:
## Imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pde import CartesianGrid, solve_laplace_equation

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_default_dtype(torch.float64)
torch.manual_seed(13);

In [ ]:
# Sampling parameters etc
n_axis = 101
n_time = 51

axis = torch.linspace(-5,5,n_axis, device=device)
time = torch.linspace(0,5,n_time, device=device)
Ps = torch.cartesian_prod(axis,time)
xx, tt = torch.meshgrid((axis, time), indexing="ij")
# Number of points
lP = Ps.shape[0]

# Get random training sets based on parameters.
# This is a mask on the first coordinate of Ps
def randomMask(n_pts = 10, initial = False):
    mask = torch.zeros(lP).bool()
    if not initial:
        mask[torch.randperm(lP)[:n_pts]] = True
    if initial:
        mask[n_time * torch.randperm(n_axis)[:n_pts]] = True
    return mask


def getVarietyPoints(base):
    x= base.unbind(1)
    t = torch.sqrt(x.square())

    return torch.stack([ torch.stack([x,t],1), torch.stack([x,-t],1) ])

def Phi(base, X):
    pts = getVarietyPoints(base)
    # return (pts.inner(X) * 1.j).exp().mean(0)
    return (pts.inner(X)).exp().mean(0)
